In [1]:
%load_ext autoreload
%autoreload 2

In [13]:
from dotenv import load_dotenv, find_dotenv
import os
load_dotenv(find_dotenv())

True

In [3]:
AA_TOKEN=os.getenv("AA_TOKEN")
NAMESPACE=os.getenv("AA_NAMESPACE")

# Document Index

## Collection

### Creating A Collection

In [4]:
from intelligence_layer.connectors import (
    CollectionPath,
    DocumentIndexClient,
)
def create_collection(collection_name: str):
    document_index = DocumentIndexClient(
            token=AA_TOKEN,
        )

    collection_path = CollectionPath(
        namespace=NAMESPACE, 
        collection=collection_name,
    )
    document_index.create_collection(collection_path)

COLLECTION_NAME="demo"
COLLECTION_NAME_DELETE="demo-collection-delete"

create_collection(COLLECTION_NAME)
create_collection(COLLECTION_NAME_DELETE)

### Get all the Collections

In [5]:
def get_collections():
    document_index = DocumentIndexClient(
            token=AA_TOKEN,
        )

    collections = document_index.list_collections(
        namespace=NAMESPACE,
    )
    return collections
collections = get_collections()
collections

[CollectionPath(namespace='rewe', collection='demo'),
 CollectionPath(namespace='rewe', collection='demo2'),
 CollectionPath(namespace='rewe', collection='demo-collection-delete'),
 CollectionPath(namespace='rewe', collection='rag-collection'),
 CollectionPath(namespace='rewe', collection='deepset')]

### Delete a Collection

In [6]:
def _delete_collection(collection_name: str):
    document_index = DocumentIndexClient(
            token=AA_TOKEN,
        )
    collection_path = CollectionPath(
        namespace=NAMESPACE, 
        collection=collection_name,
    )
    # delete collection
    document_index.delete_collection(collection_path)

_delete_collection(COLLECTION_NAME_DELETE)
# get collections to see if it was successfull
get_collections()


[CollectionPath(namespace='rewe', collection='demo'),
 CollectionPath(namespace='rewe', collection='demo2'),
 CollectionPath(namespace='rewe', collection='rag-collection'),
 CollectionPath(namespace='rewe', collection='deepset')]

## Documents

### Parse Content of Document (PDF, TXT and DOCX) 

In [10]:
from parser import FileParser

def get_file_content(path) -> tuple[str, str]:
    """Returns content and file name."""
    parser = FileParser()
    file_content, file_name = parser.parse(path)
    return file_content, file_name

PATH_PDF = "..\..\example_documents\Der-Schattenmann.pdf"
PATH_TXT = "..\..\example_documents\Plyscraper.txt"
PATH_DOCX = "..\..\example_documents\Erfunde-Programmiersprache.docx"

file_content, file_name = get_file_content(PATH_DOCX)

print("File name:", file_name, "\n", "-"*50)
print(file_content)


<>:9: SyntaxWarning: invalid escape sequence '\.'
<>:10: SyntaxWarning: invalid escape sequence '\.'
<>:11: SyntaxWarning: invalid escape sequence '\.'
<>:9: SyntaxWarning: invalid escape sequence '\.'
<>:10: SyntaxWarning: invalid escape sequence '\.'
<>:11: SyntaxWarning: invalid escape sequence '\.'
/var/folders/sr/wfpm41_x5zl1_z2xr63wxg9w0000gp/T/ipykernel_50345/93382320.py:9: SyntaxWarning: invalid escape sequence '\.'
  PATH_PDF = "..\..\example_documents\Der-Schattenmann.pdf"
/var/folders/sr/wfpm41_x5zl1_z2xr63wxg9w0000gp/T/ipykernel_50345/93382320.py:10: SyntaxWarning: invalid escape sequence '\.'
  PATH_TXT = "..\..\example_documents\Plyscraper.txt"
/var/folders/sr/wfpm41_x5zl1_z2xr63wxg9w0000gp/T/ipykernel_50345/93382320.py:11: SyntaxWarning: invalid escape sequence '\.'
  PATH_DOCX = "..\..\example_documents\Erfunde-Programmiersprache.docx"
/var/folders/sr/wfpm41_x5zl1_z2xr63wxg9w0000gp/T/ipykernel_50345/93382320.py:9: SyntaxWarning: invalid escape sequence '\.'
  PATH_PDF =

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:13                                                                                   │
│                                                                                                  │
│   10 PATH_TXT = "..\..\example_documents\Plyscraper.txt"                                         │
│   11 PATH_DOCX = "..\..\example_documents\Erfunde-Programmiersprache.docx"                       │
│   12                                                                                             │
│ ❱ 13 file_content, file_name = get_file_content(PATH_DOCX)                                       │
│   14                                                                                             │
│   15 print("File name:", file_name, "\n", "-"*50)                                                │
│   16 print(file_content)                                                                         │
│                                                                                                  │
│ in get_file_content:6                                                                            │
│                                                                                                  │
│    3 def get_file_content(path) -> tuple[str, str]:                                              │
│    4 │   """Returns content and file name."""                                                    │
│    5 │   parser = FileParser()                                                                   │
│ ❱  6 │   file_content, file_name = parser.parse(path)                                            │
│    7 │   return file_content, file_name                                                          │
│    8                                                                                             │
│    9 PATH_PDF = "..\..\example_documents\Der-Schattenmann.pdf"                                   │
│                                                                                                  │
│ /Users/moritz.gottschling/dev/aleph_alpha_tutorial_2/day_1/notebooks/parser.py:59 in parse       │
│                                                                                                  │
│   56 │   │   else:                                                                               │
│   57 │   │   │   raise Exception(f"No Parser for type {extension}")                              │
│   58 │   │                                                                                       │
│ ❱ 59 │   │   content = parser.parse(file_path)                                                   │
│   60 │   │   return content, file_name                                                           │
│   61                                                                                             │
│                                                                                                  │
│ /Users/moritz.gottschling/dev/aleph_alpha_tutorial_2/day_1/notebooks/parser.py:35 in parse       │
│                                                                                                  │
│   32 │   def __init__(self,):                                                                    │
│   33 │   │   pass                                                                                │
│   34 │   def parse(self, file_path: str) -> str:                                                 │
│ ❱ 35 │   │   doc = docx.Document(file_path)                                                      │
│   36 │   │   paragraphs = []                                                                     │
│   37 │   │   for paragraph in doc.paragraphs:                                                    │
│   38 │   │   │   paragraphs.append(paragraph.text)                                               │
│                                                            

### Upload Content

In [ ]:

from intelligence_layer.connectors import (
    CollectionPath,
    DocumentContents,
    DocumentIndexClient,
    DocumentPath,
)
from datetime import datetime

def upload_document(file_path: str, collection_name: str):
    document_index = DocumentIndexClient(
        token=AA_TOKEN,
    )
    collection_path = CollectionPath(
        namespace=NAMESPACE, 
        collection=collection_name,
    )
    file_content, file_name = get_file_content(file_path)
    document_path = DocumentPath(
            collection_path=collection_path,
            document_name=file_name,
        )
    document_contents = DocumentContents.from_text(file_content)
    document_contents.metadata = {
        "upload_date": datetime.now().isoformat()
    }
    document_index.add_document(
        document_path, 
        contents=document_contents,
    )

upload_document(PATH_PDF, COLLECTION_NAME)
upload_document(PATH_TXT, COLLECTION_NAME)
upload_document(PATH_DOCX, COLLECTION_NAME)

### Get all Documents within a Collection

In [ ]:

def get_documents(collection_name: str):
    document_index = DocumentIndexClient(
        token=AA_TOKEN,
    )
    collection_path = CollectionPath(
        namespace=NAMESPACE, 
        collection=collection_name,
    )
    documents = document_index.documents(
        collection_path=collection_path,
    )
    return documents

documents = get_documents(COLLECTION_NAME)

for document in documents:
    print(document.model_dump_json(indent=2))

### Get Single Document (and Content)

In [ ]:
from intelligence_layer.connectors import (
    CollectionPath,
    DocumentIndexClient,
    DocumentIndexRetriever,
    IndexConfiguration,
    IndexPath,
    DocumentPath
)
def get_document(collection_name: str, document_name: str):
    document_index = DocumentIndexClient(
        token=AA_TOKEN,
    )
    collection_path = CollectionPath(
        namespace=NAMESPACE, 
        collection=collection_name,
    )
    document_path = DocumentPath(
        collection_path=collection_path,
        document_name=document_name,
    )
    return document_index.document(document_path)

document_name_pdf = "Der-Schattenmann.pdf"
document_name_txt = "Plyscraper.txt"
document_name_docx = "Erfunde-Programmiersprache.docx"

document = get_document(
    collection_name=COLLECTION_NAME,
    document_name=document_name_pdf
)

print(document.model_dump_json(indent=2))

## Data Retriever

### Create Index

In [11]:
from intelligence_layer.connectors import (
    CollectionPath,
    DocumentIndexClient,
    IndexConfiguration,
    IndexPath,
)

def _create_index(index_name: str, chunk_size: int):
    document_index = DocumentIndexClient(
        token=AA_TOKEN,
    )
    index_path = IndexPath(
        namespace=NAMESPACE,
        index=index_name,
    )
    index_config = IndexConfiguration(
        embedding_type="asymmetric",
        chunk_size=chunk_size,
    )
    document_index.create_index(
        index_path=index_path,
        index_configuration=index_config,
    )
    return

INDEX_NAME = "demo-index"
chunk_size = 126
_create_index(INDEX_NAME, chunk_size)

### Get all Indexes

In [14]:
def get_all_indexes(collection_name: str):
    document_index = DocumentIndexClient(
        token=AA_TOKEN,
    )
    collection_path = CollectionPath(
        namespace=NAMESPACE, 
        collection=collection_name,
    )
    return document_index.list_assigned_index_names(collection_path)
indexes = get_all_indexes(collection_name=COLLECTION_NAME)
indexes


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /Users/moritz.gottschling/dev/aleph_alpha_tutorial_2/.venv/lib/python3.12/site-packages/intellig │
│ ence_layer/connectors/document_index/document_index.py:583 in _raise_for_status                  │
│                                                                                                  │
│   580 │                                                                                          │
│   581 │   def _raise_for_status(self, response: requests.Response) -> None:                      │
│   582 │   │   try:                                                                               │
│ ❱ 583 │   │   │   response.raise_for_status()                                                    │
│   584 │   │   except HTTPError as e:                                                             │
│   585 │   │   │   exception_factory = _status_code_to_exception.get(                             │
│   586 │   │   │   │   HTTPStatus(response.status_code), InternalError                            │
│                                                                                                  │
│ /Users/moritz.gottschling/dev/aleph_alpha_tutorial_2/.venv/lib/python3.12/site-packages/requests │
│ /models.py:1024 in raise_for_status                                                              │
│                                                                                                  │
│   1021 │   │   │   )                                                                             │
│   1022 │   │                                                                                     │
│   1023 │   │   if http_error_msg:                                                                │
│ ❱ 1024 │   │   │   raise HTTPError(http_error_msg, response=self)                                │
│   1025 │                                                                                         │
│   1026 │   def close(self):                                                                      │
│   1027 │   │   """Releases the connection back to the pool. Once this method has been            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
HTTPError: 404 Client Error:  for url: https://document-index.aleph-alpha.com/collections/rewe/demo/indexes

The above exception was the direct cause of the following exception:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:10                                                                                   │
│                                                                                                  │
│    7 │   │   collection=collection_name,                                                         │
│    8 │   )                                                                                       │
│    9 │   return document_index.list_assigned_index_names(collection_path)                        │
│ ❱ 10 indexes = get_all_indexes(collection_name=COLLECTION_NAME)                                  │
│   11 indexes                                                                                     │
│   12                                                                                             │
│                                                                                                  │
│ in get_all_indexes:9                                                                             │
│                                                                                                  │
│    6 │   │   namespace=NAMESPACE,                                                                │
│    7 │   │   collection=collection_name,                                                         │
│    8 │   )                                                                        

### Assign Index to Collection

In [ ]:

def _assign_index_to_collection(collection_name: str, index_name: str):
    document_index = DocumentIndexClient(
        token=AA_TOKEN,
    )
    collection_path = CollectionPath(
        namespace=NAMESPACE,
        collection=collection_name,
    )
    document_index.assign_index_to_collection(
        collection_path=collection_path,
        index_name=index_name
    )
    return

_assign_index_to_collection(COLLECTION_NAME, INDEX_NAME)

### Similarity Search

In [ ]:
from intelligence_layer.connectors import (
    CollectionPath,
    DocumentIndexClient,
    DocumentIndexRetriever,
)
def similariy_search(collection_name:str, index_name: str, query: str):
    document_index = DocumentIndexClient(
        token=AA_TOKEN,
    )
    
    document_index_retriever = DocumentIndexRetriever(
        document_index=document_index,
        index_name=index_name,
        namespace=NAMESPACE,
        collection=collection_name,
        k=5,
        threshold=0.5,
    )
    
    relevant_documents = document_index_retriever.get_relevant_documents_with_scores(
        query=query
    )
    return relevant_documents

query = "plyscraper"
relevant_documents = similariy_search(
    collection_name=COLLECTION_NAME,
    index_name=INDEX_NAME,
    query=query
)
relevant_documents

# Upload Data for RAG

## Upload Custom Dataset

In [ ]:
ACCEPTED_FILE_EXTENSTIONS = ["txt", "docx", "pdf"]

In [ ]:
collection_name = "rag-collection"
index_name = "rag-index"
chunk_size = 128
file_directory_path = "..\..\example_documents"

create_collection(collection_name=collection_name)
for root, dirs, files in os.walk(file_directory_path):
    for file in files:
        file_extenstion = file[file.rfind(".")+1:]
        if file_extenstion in ACCEPTED_FILE_EXTENSTIONS:
            file_path = os.path.join(root,file)
            upload_document(
                file_path=file_path,
                collection_name=collection_name
            )

_create_index(index_name=index_name, chunk_size=chunk_size)
_assign_index_to_collection(collection_name=collection_name, index_name=index_name)
get_documents(collection_name=collection_name)

## Upload quad dataset

In [ ]:
from datasets import DatasetDict, load_dataset
from typing import cast
import pandas as pd
import urllib.parse

collection_name = "rewe-workshop-prep"
index_name = "rag-index"

HF_DATASET_NAME = "deepset/germanquad"

def load_german_quad():
    dataset = load_dataset(HF_DATASET_NAME, trust_remote_code=True)
    dataset = cast(DatasetDict, dataset)
    dataset = dataset["train"]

    data = dataset.to_pandas()
    data = cast(pd.DataFrame, data)

    data = data.sample(10, random_state=4711)

    return data

def store_german_quad_in_di(data: pd.DataFrame):
    texts = data.context.unique()

    di_client = DocumentIndexClient(os.getenv("AA_TOKEN"))
    _create_index(index_name=index_name, chunk_size=chunk_size)
    _assign_index_to_collection(collection_name=collection_name, index_name=index_name)

    for text in texts:
        slug = urllib.parse.quote_plus(text[:10])
        collection_path = CollectionPath(
            namespace=NAMESPACE, 
            collection=collection_name,
        )

        document_path = DocumentPath(
            collection_path=collection_path,
            document_name=slug,
        )

        document_contents = DocumentContents(contents=[text])
        di_client.add_document(
            document_path=document_path, contents=document_contents
        )

data = load_german_quad()
store_german_quad_in_di(data)